# Crypto_Scraper

## Import 

In [3]:
import praw
import pandas as pd
from datetime import datetime
import os

In [4]:
# Authentifizierungsdetails
reddit = praw.Reddit(
    client_id='Qwk8MQZQ8YxP0iylZOcAzw',           # Deine Client ID
    client_secret='arDMtC8hLTUmWnRe1POda4W-xqj3aw',  # Dein Client Secret
    user_agent='SP3-3 Crypto Scraper/1.0 by le_remis' # User Agent
)

# Subreddit auswählen
subreddit_name = "cryptocurrency"  # Du kannst hier auch andere Subreddits angeben
subreddit = reddit.subreddit(subreddit_name)

In [5]:
print(f"Client ID: {reddit.config.client_id}")
print(f"Client Secret: {reddit.config.client_secret}")
print(f"User Agent: {reddit.config.user_agent}")


Client ID: Qwk8MQZQ8YxP0iylZOcAzw
Client Secret: arDMtC8hLTUmWnRe1POda4W-xqj3aw
User Agent: SP3-3 Crypto Scraper/1.0 by le_remis


In [6]:
try:
    subreddit = reddit.subreddit("CryptoCurrency")
    for post in subreddit.hot(limit=5):
        print(f"Title: {post.title}, Score: {post.score}")
except Exception as e:
    print(f"Fehler beim Abrufen der Subreddit-Daten: {e}")


Title: Daily Crypto Discussion - January 29, 2025 (GMT+0), Score: 31
Title: Latest Proposal• US Gov to Buy 200,000 BTC annually for 5 years • Hold for at least 20 years • Goal: Halve the U.S. national debt in 20 years, Score: 3571
Title: Men Will Live Like This Just To DCA An Extra $500 Into Crypto, Score: 2320
Title: $6,270,000,000 Worth of Seized Bitcoin Could Be Sold By UK To Pay Off Debts: Accounting Giant, Score: 165
Title: Michael Saylor Predicts Bitcoin to Hit $180,000 Amid MicroStrategy’s Record Holdings, Score: 181


## Liste der Cryptos

In [7]:
cryptos = [
    ("Ethereum", "ETH"),
    ("Ripple", "XRP"),
    ("Tether USD", "USDT"),
    ("Solana", "SOL")
]



In [8]:
subreddits = [
    "CryptoCurrency",  # Allgemeine Diskussionen über Kryptowährungen
    "CryptoMarkets",   # Diskussionen über den Kryptomarkt und Preisbewegungen
    "CryptoTrading",   # Fokus auf Trading-Strategien und Analysen
    "Altcoin",         # Diskussionen über Altcoins (alle Kryptowährungen außer Bitcoin)
    "DeFi",            # Decentralized Finance (DeFi) und Projekte
    "BitcoinBeginners",# Für Anfänger in der Krypto-Welt
    "cryptotechnology", # Fokus auf die zugrunde liegende Blockchain-Technologie
    "cryptocurrencies", # Allgemeine Diskussionen über Kryptowährungen
    "Satoshistreetsbets", # Krypto-Wetten und Spekulationen
    "Binance"        # Diskussionen über die Binance-Plattform  
]

## Scrape

In [ ]:
# Funktion, um Posts und Kommentare zu scrapen
def scrape_reddit(start_timestamp, end_timestamp, mode="initial"):
    posts = []
    comments = []
    post_ids = set()  # Set zum Speichern der Post-IDs

    for crypto_name, crypto_symbol in cryptos:
        for subreddit_name in subreddits:
            subreddit = reddit.subreddit(subreddit_name)
            print(f"Suche Posts zu {crypto_name} ({crypto_symbol}) in r/{subreddit_name}...")

            # Suche nach Namen und Kürzel
            for search_term in [crypto_name, crypto_symbol]:
                for post in subreddit.search(query=search_term, sort="new", limit=None):
                    if start_timestamp <= post.created_utc <= end_timestamp and post.id not in post_ids:
                        post_ids.add(post.id)
                        posts.append({
                            'crypto': crypto_name,
                            'search_term': search_term,
                            'subreddit': subreddit_name,
                            'post_id': post.id,
                            'title': post.title,
                            'author': str(post.author),
                            'created_utc': datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                            'score': post.score,
                            'num_comments': post.num_comments,
                            'selftext': post.selftext
                        })
                        print(f"Gefunden in r/{subreddit_name}: {post.title} (Suchbegriff: {search_term})")

                        # Kommentare sammeln
                        post.comments.replace_more(limit=0)
                        for comment in post.comments.list():
                            comments.append({
                                'post_id': post.id,
                                'comment_id': comment.id,
                                'author': str(comment.author),
                                'created_utc': datetime.utcfromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                                'score': comment.score,
                                'body': comment.body
                            })

    # In DataFrames umwandeln
    df_posts = pd.DataFrame(posts)
    df_comments = pd.DataFrame(comments)

    # DataFrames zurückgeben
    print(f"Scrape abgeschlossen: {len(df_posts)} Posts und {len(df_comments)} Kommentare gefunden.")
    return df_posts, df_comments

# **Einmaliger Scrape für das gesamte Jahr 2025**
start_of_year = int(datetime(2025, 1, 1).timestamp())  # 1. Januar 2025
now = int(datetime.now().timestamp())  # Aktuelle Zeit
print("Starte den einmaligen Scrape für das Jahr 2025...")
df_posts_initial, df_comments_initial = scrape_reddit(start_of_year, now, mode="initial")

# **Wöchentlicher Scrape**
last_week = now - 7 * 24 * 60 * 60  # 7 Tage zurück
print("Starte den wöchentlichen Scrape...")
df_posts_weekly, df_comments_weekly = scrape_reddit(last_week, now, mode="weekly")

# Beispiel: Lokale Weiterverarbeitung
print("Daten können jetzt in der Pipeline bereinigt werden...")

Starte den einmaligen Scrape für das Jahr 2025...
Suche Posts zu Ethereum (ETH) in r/CryptoCurrency...
Gefunden in r/CryptoCurrency: BlackRock Goes Live on Wormhole: Huge Win for Tokenized Assets & Ethereum (ETH) Ecosystem (Suchbegriff: Ethereum)
Gefunden in r/CryptoCurrency: Paradigm commits $1.25 million to aid developer of Ethereum privacy protocol, Tornado Cash, in continued legal battle (Suchbegriff: Ethereum)
Gefunden in r/CryptoCurrency: Any advice for someone relatively new to crypto who has 40k to enter in with? (Suchbegriff: Ethereum)
Gefunden in r/CryptoCurrency: World Liberty Financial buys the dip on Ethereum (ETH), extending a week of accumulation (Suchbegriff: Ethereum)
Gefunden in r/CryptoCurrency: Ethereum Foundation Sells Another 100 ETH, But There's Still 'Hopium' For Holders (Suchbegriff: Ethereum)
Gefunden in r/CryptoCurrency: Bullish Signal? Ethereum Sees Highest Surge in Network Growth Since October 2022 (Suchbegriff: Ethereum)
Gefunden in r/CryptoCurrency: Ether

In [10]:
df_posts_initial.head()

,crypto,search_term,subreddit,post_id,title,author,created_utc,score,num_comments,selftext
0,Ethereum,Ethereum,CryptoCurrency,1icqgk5,BlackRock Goes Live on Wormhole: Huge Win for ...,kirtash93,2025-01-29 10:00:24,6,30,Today I crossed with this another bullish news...
1,Ethereum,Ethereum,CryptoCurrency,1icobde,Paradigm commits $1.25 million to aid develope...,aminok,2025-01-29 07:14:30,7,2,
2,Ethereum,Ethereum,CryptoCurrency,1ichty9,Any advice for someone relatively new to crypt...,LessMandmoreP,2025-01-29 01:07:32,0,115,"Firstly, no I’m not throwing my entire savings..."
3,Ethereum,Ethereum,CryptoCurrency,1ic6mf3,World Liberty Financial buys the dip on Ethere...,KIG45,2025-01-28 17:15:02,186,47,
4,Ethereum,Ethereum,CryptoCurrency,1ibw0wk,"Ethereum Foundation Sells Another 100 ETH, But...",DaRunningdead,2025-01-28 07:07:19,0,15,


## Datenbereinigung 

In [16]:
def clean_data(df_posts, df_comments):
    # 1. Duplikate entfernen
    df_posts = df_posts.drop_duplicates(subset=["post_id"])
    df_comments = df_comments.drop_duplicates(subset=["comment_id"])
    
    # 2. Fehlende Werte behandeln
    df_posts['selftext'] = df_posts['selftext'].fillna('')  # Fehlende Posttexte auffüllen
    df_comments['body'] = df_comments['body'].fillna('')  # Fehlende Kommentare auffüllen
    
    # 3. Zeitstempel konvertieren
    df_posts['created_utc'] = pd.to_datetime(df_posts['created_utc'])
    df_comments['created_utc'] = pd.to_datetime(df_comments['created_utc'])
    
    # 4. Filterung nach Qualität (z. B. Spam entfernen)
    df_posts = df_posts[df_posts['score'] > 0]  # Posts mit negativem Score entfernen
    df_comments = df_comments[df_comments['score'] > 0]  # Kommentare mit negativem Score entfernen
    
    return df_posts, df_comments


Funktion zur bereinigung auf die Daten anwenden

In [17]:
# Bereinigen der Daten
df_posts_clean, df_comments_clean = clean_data(df_posts_initial, df_comments_initial)

# Überprüfen, wie viele Einträge übrig sind
print(f"Bereinigte Posts: {len(df_posts_clean)}")
print(f"Bereinigte Kommentare: {len(df_comments_clean)}")

Bereinigte Posts: 550
Bereinigte Kommentare: 27440


## Google Drive Export

Pfad

In [27]:
# Definiere den Speicherpfad zu deinem lokalen Google Drive
DRIVE_PATH = "G:/Meine Ablage/"


Export Funktion

In [29]:
def append_to_csv(df_new, filename):
    file_path = os.path.join(DRIVE_PATH, filename)
    
    # Falls die Datei bereits existiert, lese sie ein und hänge die neuen Daten an
    if os.path.exists(file_path):
        df_existing = pd.read_csv(file_path)
        df_combined = pd.concat([df_existing, df_new], ignore_index=True)
    else:
        df_combined = df_new  # Falls die Datei nicht existiert, erstelle sie neu
    
    # CSV überschreiben mit neuen + alten Daten
    df_combined.to_csv(file_path, index=False, encoding="utf-8")
    print(f"Datei erfolgreich aktualisiert: {file_path}")

def export_to_drive(df_posts, df_comments):
    try:
        # Posts an bestehende Datei anhängen
        append_to_csv(df_posts, "gold_clean_posts.csv")
        # Kommentare an bestehende Datei anhängen
        append_to_csv(df_comments, "gold_clean_comments.csv")

    except Exception as e:
        print(f"Fehler beim Export: {e}")

Export Funnktion anwenden

In [28]:
# Export der bereinigten Daten
export_to_drive(df_posts_clean, df_comments_clean)

Posts erfolgreich exportiert nach: G:/Meine Ablage/gold_clean_posts.csv
Kommentare erfolgreich exportiert nach: G:/Meine Ablage/gold_clean_comments.csv
